In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from persim import PersImage
import gudhi
import os
import glob
from matplotlib import ticker, cm
from matplotlib.colors import LogNorm
from sklearn.cluster import KMeans
import time
from sklearn.decomposition import PCA
import matplotlib as mpl
from sklearn import preprocessing

load specific PD for tests

In [3]:
temp=np.loadtxt("condorSims/Data_2d_Ising_Wolff_N=50/2.25/1551766531PD1.dat")

OSError: condor/Data_2d_Ising_Wolff_N=50/2.49/1550489717PD1.dat not found.

Define persistence image function... this can likely be sped up

In [2]:
#pdlocnum has format [[birth1, persistence1, counts1],...]
#bounds should be a list [bmin,bmax,pmin,pmax]
#bins should be a list [bbins, pbins]
def PerIm(pdlocnum,bounds,bins,stdev,res=1000):
    x = np.linspace(bounds[0], bounds[1], res)
    y = np.linspace(bounds[2], bounds[3], res)
    xx, yy = np.meshgrid(x, y, sparse=True)
    z=np.zeros([res,res])
    for i in np.arange(len(pdlocnum)):
        z=z+np.log(1+pdlocnum[i][1])*pdlocnum[i][2]*np.exp((-0.5*(xx-pdlocnum[i][0])**2-0.5*(yy-pdlocnum[i][1])**2)*stdev**-2)
    return z.reshape(bins[0],res//bins[0],bins[1],res//bins[1]).sum(3).sum(1)

In [747]:
def PerImAlt(pdlocnum,bounds,bins,stdev,res=1000):
    x = np.linspace(bounds[0], bounds[1], res)
    y = np.linspace(bounds[2], bounds[3], res)
    xx, yy = np.meshgrid(x, y, sparse=True)
    z=np.sum(np.array([np.log(1+elm[1])*elm[2]*np.exp((-0.5*(xx-elm[0])**2-0.5*(yy-elm[1])**2)*stdev**-2) for elm in pdlocnum]),axis=0)
    return z.reshape(bins[0],res//bins[0],bins[1],res//bins[1]).sum(3).sum(1)

try it out on test PD

In [750]:
bounds=[0,15.5,0,13]
bins=[10,10]
res=bins[0]*10
x = np.linspace(bounds[0], bounds[1], bins[0])
y = np.linspace(bounds[2], bounds[3], bins[1])
t1=time.time()
z=PerIm(temp,bounds,bins,1,res=res)
t2=time.time()
z=PerImAlt(temp,bounds,bins,1,res=res)
t3=time.time()
#fig, ax = plt.subplots()
#cs=ax.imshow(np.log(1+z),origin='lower')
#cs = ax.contourf(x, y, z,locator=ticker.LogLocator())
#locator=ticker.LogLocator()
#cbar = fig.colorbar(cs)
print((t2-t1,t3-t2))

(0.00792694091796875, 0.010153055191040039)


PerIm is faster than PerImAlt

In [94]:
np.sort(os.listdir('condorSims/Data_2d_Ising_Wolff_N=50'))

array(['1.0', '1.05', '1.1', '1.15', '1.2', '1.25', '1.3', '1.35', '1.4',
       '1.45', '1.5', '1.55', '1.6', '1.65', '1.7', '1.75', '1.8', '1.85',
       '1.9', '1.95', '2.0', '2.05', '2.1', '2.15', '2.2', '2.25', '2.3',
       '2.35', '2.4', '2.45', '2.5', '2.55', '2.6', '2.65', '2.7', '2.75',
       '2.8', '2.85', '2.9', '2.95', '3.0', '3.05', '3.1', '3.15', '3.2',
       '3.25', '3.3', '3.35', '3.4', '3.45', '3.5'], dtype='<U4')

# Compute, save PDs for $\alpha$ filtrations

There could be a more efficient way to do the below. E.g. the extra check for infinity is inefficient, should remove at the end. Also it's likely faster to load all files at once, etc.

In [107]:
Tlist=[]
PD0list=[]
PD1list=[]
flist=np.sort(glob.glob("condorSims/Dat*/*/*.txt"))
#flistAlt=np.append(flist[0:10],flist[-11:-1])
for f in flist[:9000]:
#for f in flistAlt:
    bounds=[x for x,y in enumerate(f) if y == '/']
    #Tlist.append(float(f[bounds[1]+1:bounds[2]]))
    data = np.loadtxt(f, dtype=int)
    alpha_complex=gudhi.AlphaComplex(data)
    simplex_tree=alpha_complex.create_simplex_tree()
    PD=simplex_tree.persistence()
    PD0=[]
    PD1=[]
    for elm in PD:
        if elm[0]==1:
            PD1.append((elm[1][0],elm[1][1]-elm[1][0]))
        else:
            if elm[1][1]!=float('inf'):
                PD0.append((elm[1][0],elm[1][1]-elm[1][0]))
    PD0=np.round(PD0,decimals=5)
    PD1=np.round(PD1,decimals=5)
    loc,counts=np.unique(PD0,axis=0,return_counts=True)
    toSave=np.array([[loc[i][0],loc[i][1],counts[i]] for i in np.arange(len(counts))])
    np.savetxt(f[:len(f)-4]+"PD0.dat",toSave)
    loc,counts=np.unique(PD1,axis=0,return_counts=True)
    toSave=np.array([[loc[i][0],loc[i][1],counts[i]] for i in np.arange(len(counts))])
    np.savetxt(f[:len(f)-4]+"PD1.dat",toSave)
    print(f)

condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551745319.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551745341.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551745363.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551745385.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551745407.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551745429.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551745450.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551745472.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551745495.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551745519.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551745545.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551745568.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551745596.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551745622.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551745646.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551745669.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551745694.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551745717.txt
condorSims

condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551746675.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551746681.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551746687.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551746694.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551746695.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551746701.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551746707.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551746715.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551746719.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551746730.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551746732.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551746735.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551746742.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551746753.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551746754.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551746758.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551746765.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551746767.txt
condorSims

condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551802438.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551802444.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551802446.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551802451.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551802453.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551802455.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551802456.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551802459.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551802460.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551802461.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551802463.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551802470.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551802473.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551802476.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551802477.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551802478.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551802479.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551802480.txt
condorSims

condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551843795.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551843817.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551843839.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551843860.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551843881.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551843903.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551843926.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551843947.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551843969.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551843990.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551844012.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551844034.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551844056.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551844077.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551844099.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551844121.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551844143.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551844166.txt
condorSims

condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551847011.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551847033.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551847056.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551847078.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551847100.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551847122.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551847144.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551847167.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551847189.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551847210.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551847234.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551847256.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551847277.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551847297.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551847318.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551847340.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551847360.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.0/1551847382.txt
condorSims

...

condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551753475.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551753498.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551753520.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551753544.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551753567.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551753590.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551753603.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551753613.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551753627.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551753637.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551753651.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551753660.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551753676.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551753682.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551753699.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551753705.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551753722.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551753728.txt
condorSims

condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551803795.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551803799.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551803801.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551803803.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551803806.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551803807.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551803809.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551803810.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551803817.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551803819.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551803823.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551803825.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551803826.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551803827.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551803828.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551803831.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551803833.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551803841.txt
condorSims

condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551843616.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551843639.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551843663.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551843689.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551843715.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551843739.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551843758.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551843784.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551843806.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551843827.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551843850.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551843874.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551843897.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551843916.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551843933.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551843956.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551843983.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551844005.txt
condorSims

condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551847208.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551847227.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551847245.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551847266.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551847291.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551847313.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551847339.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551847363.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551847387.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551847413.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551847438.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551847461.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551847486.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551847510.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551847534.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551847560.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551847585.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551847610.txt
condorSims

condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551850876.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551850900.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551850925.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551850947.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551850970.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551850997.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551851022.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551851046.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551851071.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551851097.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551851120.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551851146.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551851171.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551851195.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551851219.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551851244.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551851267.txt
condorSims/Data_2d_Ising_Wolff_N=50/1.3/1551851290.txt
condorSims

In [105]:
flist[38500]

'condorSims/Data_2d_Ising_Wolff_N=50/2.4/1551773493.txt'

Potentially speedier version...

In [752]:
Tlist=[]
PD0list=[]
PD1list=[]
flist=np.sort(glob.glob("*Ising*/*/*.txt"))
flistAlt=np.append(flist[0:100],flist[-100:])
#for f in flistAlt:
bounds=[[x for x,y in enumerate(f) if y == '/'] for f in flistAlt]
Tlist=[float(flistAlt[i][bounds[i][0]+1:bounds[i][1]]) for i in np.arange(len(flistAlt))]
print('loading data')
datas = [np.loadtxt(f, dtype=int) for f in flistAlt]
print('calculating alpha complexes')
alpha_complexes=[gudhi.AlphaComplex(data) for data in datas]
simplex_trees=[alpha_complex.create_simplex_tree() for alpha_complex in alpha_complexes]
print('calculating persistences')
PD=[simplex_tree.persistence() for simplex_tree in simplex_trees]
'''   PD0=[]
    PD1=[]
    for elm in PD:
        if elm[0]==1:
            PD1.append((elm[1][0],elm[1][1]-elm[1][0]))
        else:
            if elm[1][1]!=float('inf'):
                PD0.append((elm[1][0],elm[1][1]-elm[1][0]))
    PD0=np.round(PD0,decimals=5)
    PD1=np.round(PD1,decimals=5)
    loc,counts=np.unique(PD0,axis=0,return_counts=True)
    toSave=np.array([[loc[i][0],loc[i][1],counts[i]] for i in np.arange(len(counts))])
    np.savetxt(f[:len(f)-4]+"PD0.dat",toSave)
    loc,counts=np.unique(PD1,axis=0,return_counts=True)
    toSave=np.array([[loc[i][0],loc[i][1],counts[i]] for i in np.arange(len(counts))])
    np.savetxt(f[:len(f)-4]+"PD1.dat",toSave)
    print(f)'''

loading data
calculating alpha complexes
calculating persistences


'\'   PD0=[]\n    PD1=[]\n    for elm in PD:\n        if elm[0]==1:\n            PD1.append((elm[1][0],elm[1][1]-elm[1][0]))\n        else:\n            if elm[1][1]!=float(\'inf\'):\n                PD0.append((elm[1][0],elm[1][1]-elm[1][0]))\n    PD0=np.round(PD0,decimals=5)\n    PD1=np.round(PD1,decimals=5)\n    loc,counts=np.unique(PD0,axis=0,return_counts=True)\n    toSave=np.array([[loc[i][0],loc[i][1],counts[i]] for i in np.arange(len(counts))])\n    np.savetxt(f[:len(f)-4]+"PD0.dat",toSave)\n    loc,counts=np.unique(PD1,axis=0,return_counts=True)\n    toSave=np.array([[loc[i][0],loc[i][1],counts[i]] for i in np.arange(len(counts))])\n    np.savetxt(f[:len(f)-4]+"PD1.dat",toSave)\n    print(f)'

In [794]:
PD0s=[[index for index,value in enumerate(elm) if (value[0] ==0)] for elm in PD]
d=np.array([PD[0][t][1] for t in tt])
np.unique(d,axis=0,return_counts=True)

(array([[0.  , 0.25],
        [0.  ,  inf]]), array([2475,    1]))